In [1]:
import os
import glob
import openpyxl
import pyperclip
import time
import pyautogui
import re
from datetime import datetime
from datetime import timedelta
import subprocess
import pandas as pd
import gspread
from gspread_pandas import Spread, Client
import warnings
warnings.simplefilter("ignore")

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


credentials_path = r"C:\Working_customore\google_secret.json"

# Define the scope
scope = ["https://spreadsheets.google.com/feeds",
          "https://www.googleapis.com/auth/drive"]

# Add your credentials file
creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)

# Authorize the client
client = gspread.authorize(creds)

# Open Google Sheet by name
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1qQ5JSuvTGJzOIoLAHKN0INHgrgB_LG6NiDPKTOR6lLw/edit?gid=1007344023#gid=1007344023' #input the working file name
spreadsheet = client.open_by_url(spreadsheet_url)

In [3]:
import os
from gspread_pandas import Spread

# Set the path to your Google credentials JSON file
os.environ['GSPREAD_PANDAS_CONFIG_DIR'] = r"C:\Working_customore" #input link of folder in local to JSON key file

# Continue with your existing code
spread = Spread('2024-09-09_[working] reckitt_benckiser_hourly sales tracking [Phúc]')  #input the working file name

In [4]:
#check what is the cordination for the mouse to click

import pyautogui
import time

# Function to get coordinates after a delay and run for a specific duration
def get_coordinates(delay_seconds, duration_seconds):
    try:
        print("Move your mouse to the desired location within the next 30 seconds.")
        time.sleep(delay_seconds)

        start_time = time.time()
        while (time.time() - start_time) < duration_seconds:
            x, y = pyautogui.position()
            print(f"X: {x}, Y: {y}")
            time.sleep(1)  # Update coordinates every 1 second

    except KeyboardInterrupt:
        print("Coordinates stopped.")

if __name__ == "__main__":
    # Delay for 60 seconds before starting, and run for 30 seconds
    get_coordinates(delay_seconds=10, duration_seconds=30)


Move your mouse to the desired location within the next 30 seconds.
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301
X: 1298, Y: 301


In [6]:
# Calculate the initial time to start
start_time = datetime.now().replace(hour=17, minute=11, second=0, microsecond=0)  # Please minus 2 mins vs real time you want to start

# Calculate the end time (5:00 PM)
end_time = start_time.replace(hour=17, minute=19, second=0)  # Real time you want to end

# Specify the folder path where the files are downloaded
folder_path = r"C:\Working_customore\live tracker 10-10"

# Wait until the start time is reached
while datetime.now() < start_time:
    time.sleep(30)  # Check every minute if it's time to start

while datetime.now() < end_time:
    # Calculate the time until the next click should be taken (10:35 AM, 11:35 AM, etc.)
    now = datetime.now()
    next_screenshot_time = now.replace(microsecond=0) + timedelta(seconds=60)

    # Replace these coordinates with the actual coordinates of the button
    button_x, button_y = 1298, 301
    pyautogui.click(button_x, button_y)
    print('Click at', datetime.now())
    time.sleep(20)  # Wait 20 secs for report finished downloading

    # Function to identify the latest Excel file based on the naming convention
    def identify_latest_excel_file(folder_path):
        latest_files_by_date = {}
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".xlsx"):
                match = re.match(r"^Brand_Portal-Business_Insights---Real_Time_Analysis---Real_Time_Sales-Gross_Data-(\d{4}.\d{2}.\d{2})(?:\s*\((\d*)\))?.xlsx$", file_name)
                if match:
                    date_str, number_str = match.groups()
                    file_date = datetime.strptime(date_str, "%Y.%m.%d")
                    file_number = int(number_str) if number_str else 0
                    if file_date not in latest_files_by_date:
                        latest_files_by_date[file_date] = []
                    latest_files_by_date[file_date].append((file_number, os.path.join(folder_path, file_name)))
        
        if not latest_files_by_date:
            return None
        
        latest_date = max(latest_files_by_date.keys())
        latest_files = latest_files_by_date[latest_date]
        
        if len(latest_files) > 1:
            latest_file = max(latest_files, key=lambda x: x[0])
        else:
            latest_file = latest_files[0]
        
        return latest_file[1]

    # Main execution
    if __name__ == "__main__":
        folder_path = r"C:\Working_customore\live tracker 10-10"

        # Identify the latest Excel file based on the naming convention
        latest_file = identify_latest_excel_file(folder_path)
        
        if latest_file:
            try:
                print(f"Reading file: {latest_file}")
                text = pd.read_excel(latest_file, sheet_name='Shop Ranking', header=None)
                text.insert(0, 'Updated time', "")
                text['Updated time'] = now.strftime("%Y-%m-%d %H:%M:%S")
                real_time_report = Spread('2024-09-09_[working] reckitt_benckiser_hourly sales tracking [Phúc]')

                worksheet = spreadsheet.worksheet('SHP_BP2')
                # Determine the last row with data
                existing_data = worksheet.get_all_values()
                last_row = len(existing_data) + 1

                # Write new data to Google Sheet below the last row
                real_time_report.df_to_sheet(text, sheet='SHP_BP2', start=f'A{last_row}', index=False, headers=False)

                print(f"Uploaded data from {latest_file} to Google Sheets.")
            except Exception as e:
                print(f"Error processing file {latest_file}: {e}")
        else:
            print("No Excel files found in the specified folder.")

        # Sleep until the next screenshot time
        sleep_duration = (next_screenshot_time - now).total_seconds()
        if sleep_duration > 0:
            time.sleep(sleep_duration)
                    
print("Done auto click")

Click at 2024-10-03 17:11:24.289310
Reading file: C:\Working_customore\live tracker 10-10\Brand_Portal-Business_Insights---Real_Time_Analysis---Real_Time_Sales-Gross_Data-2024.10.03 (6).xlsx
Uploaded data from C:\Working_customore\live tracker 10-10\Brand_Portal-Business_Insights---Real_Time_Analysis---Real_Time_Sales-Gross_Data-2024.10.03 (6).xlsx to Google Sheets.
Click at 2024-10-03 17:12:51.585888


KeyboardInterrupt: 

In [1]:
latest_file

NameError: name 'latest_file' is not defined